In [3]:


import os
import requests

BOT_TOKEN = os.getenv("TG_BOT_TOKEN")   # 깃허브 Secrets에서 불러옴
CHAT_ID   = os.getenv("TG_CHAT_ID")     # 깃허브 Secrets에서 불러옴


msg = "텔레그램 연결 테스트 🚀"
url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage"
res = requests.get(url, params={"chat_id": CHAT_ID, "text": msg})

print(res.status_code, res.text)

200 {"ok":true,"result":{"message_id":2,"from":{"id":7242833303,"is_bot":true,"first_name":"Botforjs","username":"JSquantbot"},"chat":{"id":8196864798,"first_name":"\uc9c0\uc218","last_name":"\ubc15","type":"private"},"date":1759201790,"text":"\ud154\ub808\uadf8\ub7a8 \uc5f0\uacb0 \ud14c\uc2a4\ud2b8 \ud83d\ude80"}}


In [13]:
# 11종목 한정 MACD+OBV 시그널 → 텔레그램 알림 (에러패치 포함)
# 필요: pip install yfinance pandas numpy requests

import os, time, datetime as dt
import numpy as np
import pandas as pd
import yfinance as yf
import requests

# ====== 설정 ======
TICKERS = ["AAPL","MSFT","NVDA","AMZN","GOOGL","META","BRK-B","AVGO","TSLA","XOM","PLTR"]

BOT_TOKEN = "7242833303:AAGMw_2WmhVbPcjScXvrDfkZpzT02F7NbT4"   # BotFather 토큰
CHAT_ID   = 8196864798     # 개인(양수) 또는 그룹(-100...) chat_id

MACD_SHORT, MACD_LONG, MACD_SIGNAL = 12, 26, 9
OBV_SMA_N = 5
USE_PRICE_ABOVE_SMA50 = True
PERIOD = "8mo"

# ====== 텔레그램 알림 ======
def notify(text: str):
    if not (BOT_TOKEN and CHAT_ID):
        print("[SKIP] Telegram not configured."); return
    url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage"
    res = requests.get(url, params={"chat_id": CHAT_ID, "text": text})
    print("Telegram status:", res.status_code)

# ====== 지표 계산 ======
def compute_indicators(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty or len(df) < 50:
        return pd.DataFrame(index=df.index)

    # ★ 핵심 패치: 반드시 1차원 Series로 변환
    close = df["Close"].squeeze().astype(float)
    vol   = df["Volume"].squeeze().astype(float)

    ema12 = close.ewm(span=MACD_SHORT, adjust=False).mean()
    ema26 = close.ewm(span=MACD_LONG , adjust=False).mean()
    macd  = ema12 - ema26
    sig   = macd.ewm(span=MACD_SIGNAL, adjust=False).mean()
    hist  = macd - sig

    delta = close.diff()
    sign  = np.where(delta > 0, 1, np.where(delta < 0, -1, 0))
    obv   = (sign * vol.fillna(0)).cumsum()
    obv_sma = obv.rolling(OBV_SMA_N, min_periods=OBV_SMA_N).mean()

    sma50 = close.rolling(50, min_periods=50).mean()

    out = pd.DataFrame({
        "Close": close,
        "MACD_HIST": hist,
        "OBV": obv,
        "OBV_SMA": obv_sma,
        "SMA50": sma50
    }, index=df.index)
    return out.dropna()

def is_signal(today, yesterday) -> bool:
    EPS = 1e-3          # 아주 작은 허용오차(선택적으로 완화)

    # MACD: 교차(어제≤0 & 오늘>0) OR 오늘>0 만으로도 통과
    macd_ok = (today["MACD_HIST"] > 0) or (
        (yesterday["MACD_HIST"] <= 0) and (today["MACD_HIST"] > 0)
    )

    # OBV: '>'를 '>='로 완화 + 0.5% 여유 허용
    obv_ok = today["OBV"] >= today["OBV_SMA"] * 0.995

    # 가격: SMA50 위 조건을 0.5% 여유로 완화
    price_ok = today["Close"] >= today["SMA50"] * 0.995

    return bool(macd_ok and obv_ok and price_ok)


# ====== 메인 ======
def main():
    sigs = []
    for t in TICKERS:
        try:
            df = yf.download(t, period=PERIOD, interval="1d", auto_adjust=False)
            if df.empty:
                continue
            ind = compute_indicators(df)
            if ind.empty or len(ind) < 2:
                continue
            yest, today = ind.iloc[-2], ind.iloc[-1]
            if is_signal(today, yest):
                sigs.append(t)
            time.sleep(0.2)  # API 호출 간 대기
        except Exception as e:
            print(f"[ERROR] {t}: {e}")

    trade_date = dt.date.today().strftime("%Y-%m-%d")
    if sigs:
        msg = f"[{trade_date}] 시그널 종목: {', '.join(sigs)}"
    else:
        msg = f"[{trade_date}] 시그널 없음"

    print(msg)      # 콘솔 출력
    notify(msg)     # 텔레그램 전송

if __name__ == "__main__":
    main()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[2025-09-30] 시그널 종목: MSFT, NVDA, TSLA, XOM
Telegram status: 200
